# Feature Engineering: Bureau Data

Aggregate bureau.csv into applicant-level features.

**Acceptance Criteria:**
- Aggregations per SK_ID_CURR
- Credit count and amount statistics
- Clean merge with main table

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

## 1. Load and Explore Bureau Data

In [ ]:
bureau = pd.read_csv('../data/raw/bureau.csv')
print(f"Bureau shape: {bureau.shape}")
bureau.head()

In [3]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [4]:
bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.610875e+06,1.082775e+06,5.919400e+05,1.716428e+06,1.716415e+06,1.458759e+06,1.124648e+06,1.716428e+06,1.716428e+06,4.896370e+05
mean,2.782149e+05,5.924434e+06,-1.142108e+03,8.181666e-01,5.105174e+02,-1.017437e+03,3.825418e+03,6.410406e-03,3.549946e+05,1.370851e+05,6.229515e+03,3.791276e+01,-5.937483e+02,1.571276e+04
std,1.029386e+05,5.322657e+05,7.951649e+02,3.654443e+01,4.994220e+03,7.140106e+02,2.060316e+05,9.622391e-02,1.149811e+06,6.774011e+05,4.503203e+04,5.937650e+03,7.207473e+02,3.258269e+05
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.206000e+04,-4.202300e+04,0.000000e+00,0.000000e+00,0.000000e+00,-4.705600e+06,-5.864061e+05,0.000000e+00,-4.194700e+04,0.000000e+00
25%,1.888668e+05,5.463954e+06,-1.666000e+03,0.000000e+00,-1.138000e+03,-1.489000e+03,0.000000e+00,0.000000e+00,5.130000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-9.080000e+02,0.000000e+00
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.300000e+02,-8.970000e+02,0.000000e+00,0.000000e+00,1.255185e+05,0.000000e+00,0.000000e+00,0.000000e+00,-3.950000e+02,0.000000e+00
75%,3.674260e+05,6.385681e+06,-4.740000e+02,0.000000e+00,4.740000e+02,-4.250000e+02,0.000000e+00,0.000000e+00,3.150000e+05,4.015350e+04,0.000000e+00,0.000000e+00,-3.300000e+01,1.350000e+04
max,4.562550e+05,6.843457e+06,0.000000e+00,2.792000e+03,3.119900e+04,0.000000e+00,1.159872e+08,9.000000e+00,5.850000e+08,1.701000e+08,4.705600e+06,3.756681e+06,3.720000e+02,1.184534e+08


In [5]:
# Check unique applicants vs total records
print(f"Unique applicants (SK_ID_CURR): {bureau['SK_ID_CURR'].nunique()}")
print(f"Total bureau records: {len(bureau)}")
print(f"Average records per applicant: {len(bureau) / bureau['SK_ID_CURR'].nunique():.2f}")

Unique applicants (SK_ID_CURR): 305811
Total bureau records: 1716428
Average records per applicant: 5.61


## 2. Aggregate Numeric Features per SK_ID_CURR

In [6]:
# Credit count and amount statistics
bureau_agg = bureau.groupby('SK_ID_CURR').agg(
    # Count statistics
    bureau_credit_count=('SK_ID_BUREAU', 'count'),
    
    # Credit amount statistics
    bureau_credit_sum_mean=('AMT_CREDIT_SUM', 'mean'),
    bureau_credit_sum_sum=('AMT_CREDIT_SUM', 'sum'),
    bureau_credit_sum_max=('AMT_CREDIT_SUM', 'max'),
    bureau_credit_sum_min=('AMT_CREDIT_SUM', 'min'),
    
    # Debt statistics
    bureau_debt_sum=('AMT_CREDIT_SUM_DEBT', 'sum'),
    bureau_debt_mean=('AMT_CREDIT_SUM_DEBT', 'mean'),
    bureau_debt_max=('AMT_CREDIT_SUM_DEBT', 'max'),
    
    # Overdue statistics
    bureau_overdue_max=('AMT_CREDIT_MAX_OVERDUE', 'max'),
    bureau_overdue_mean=('AMT_CREDIT_MAX_OVERDUE', 'mean'),
    bureau_days_overdue_max=('CREDIT_DAY_OVERDUE', 'max'),
    
    # Days/time statistics
    bureau_days_credit_mean=('DAYS_CREDIT', 'mean'),
    bureau_days_credit_min=('DAYS_CREDIT', 'min'),
    bureau_days_credit_max=('DAYS_CREDIT', 'max'),
    
    # Credit prolongation
    bureau_prolong_sum=('CNT_CREDIT_PROLONG', 'sum'),
    
    # Annuity
    bureau_annuity_mean=('AMT_ANNUITY', 'mean'),
    bureau_annuity_sum=('AMT_ANNUITY', 'sum'),
).reset_index()

print(f"Aggregated shape: {bureau_agg.shape}")
bureau_agg.head()

Aggregated shape: (305811, 18)


,SK_ID_CURR,bureau_credit_count,bureau_credit_sum_mean,bureau_credit_sum_sum,bureau_credit_sum_max,bureau_credit_sum_min,bureau_debt_sum,bureau_debt_mean,bureau_debt_max,bureau_overdue_max,bureau_overdue_mean,bureau_days_overdue_max,bureau_days_credit_mean,bureau_days_credit_min,bureau_days_credit_max,bureau_prolong_sum,bureau_annuity_mean,bureau_annuity_sum
0,100001,7,207623.571429,1453365.000,378000.0,85500.0,596686.5,85240.928571,373239.0,NaN,NaN,0,-735.000000,-1572,-49,0,3545.357143,24817.5
1,100002,8,108131.945625,865055.565,450000.0,0.0,245781.0,49156.200000,245781.0,5043.645,1681.029,0,-874.000000,-1437,-103,0,0.000000,0.0
2,100003,4,254350.125000,1017400.500,810000.0,22248.0,0.0,0.000000,0.0,0.000,0.000,0,-1400.750000,-2586,-606,0,NaN,0.0
3,100004,2,94518.900000,189037.800,94537.8,94500.0,0.0,0.000000,0.0,0.000,0.000,0,-867.000000,-1326,-408,0,NaN,0.0
4,100005,3,219042.000000,657126.000,568800.0,29826.0,568408.5,189469.500000,543087.0,0.000,0.000,0,-190.666667,-373,-62,0,1420.500000,4261.5


## 3. Aggregate Categorical Features

In [7]:
# Count of active vs closed credits
print("Credit Active values:")
print(bureau['CREDIT_ACTIVE'].value_counts())

Credit Active values:
CREDIT_ACTIVE
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: count, dtype: int64


In [8]:
# Pivot credit active status
active_counts = bureau.groupby(['SK_ID_CURR', 'CREDIT_ACTIVE']).size().unstack(fill_value=0)
active_counts.columns = ['bureau_' + col.lower().replace(' ', '_') for col in active_counts.columns]
active_counts = active_counts.reset_index()

active_counts.head()

,SK_ID_CURR,bureau_active,bureau_bad_debt,bureau_closed,bureau_sold
0,100001,3,0,4,0
1,100002,2,0,6,0
2,100003,1,0,3,0
3,100004,0,0,2,0
4,100005,2,0,1,0


In [9]:
# Merge active counts with main aggregation
bureau_agg = bureau_agg.merge(active_counts, on='SK_ID_CURR', how='left')
print(f"Shape after adding active counts: {bureau_agg.shape}")

Shape after adding active counts: (305811, 22)


In [10]:
# Count of credit types
print("Credit Type values:")
print(bureau['CREDIT_TYPE'].value_counts())

Credit Type values:
CREDIT_TYPE
Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: count, dtype: int64


In [11]:
# Pivot credit types (top categories only to avoid too many columns)
credit_type_counts = bureau.groupby(['SK_ID_CURR', 'CREDIT_TYPE']).size().unstack(fill_value=0)
credit_type_counts.columns = ['bureau_type_' + col.lower().replace(' ', '_').replace(',', '') for col in credit_type_counts.columns]
credit_type_counts = credit_type_counts.reset_index()

# Merge credit type counts
bureau_agg = bureau_agg.merge(credit_type_counts, on='SK_ID_CURR', how='left')
print(f"Shape after adding credit types: {bureau_agg.shape}")

Shape after adding credit types: (305811, 37)


## 4. Create Derived Features (Ratios)

In [12]:
# Debt to credit ratio
bureau_agg['bureau_debt_credit_ratio'] = bureau_agg['bureau_debt_sum'] / bureau_agg['bureau_credit_sum_sum']

# Active credit ratio
if 'bureau_active' in bureau_agg.columns and 'bureau_closed' in bureau_agg.columns:
    bureau_agg['bureau_active_ratio'] = bureau_agg['bureau_active'] / bureau_agg['bureau_credit_count']

# Replace inf with NaN
bureau_agg = bureau_agg.replace([np.inf, -np.inf], np.nan)

bureau_agg.head()

,SK_ID_CURR,bureau_credit_count,bureau_credit_sum_mean,bureau_credit_sum_sum,bureau_credit_sum_max,bureau_credit_sum_min,bureau_debt_sum,bureau_debt_mean,bureau_debt_max,bureau_overdue_max,...,bureau_type_loan_for_purchase_of_shares_(margin_lending),bureau_type_loan_for_the_purchase_of_equipment,bureau_type_loan_for_working_capital_replenishment,bureau_type_microloan,bureau_type_mobile_operator_loan,bureau_type_mortgage,bureau_type_real_estate_loan,bureau_type_unknown_type_of_loan,bureau_debt_credit_ratio,bureau_active_ratio
0,100001,7,207623.571429,1453365.000,378000.0,85500.0,596686.5,85240.928571,373239.0,NaN,...,0,0,0,0,0,0,0,0,0.410555,0.428571
1,100002,8,108131.945625,865055.565,450000.0,0.0,245781.0,49156.200000,245781.0,5043.645,...,0,0,0,0,0,0,0,0,0.284122,0.250000
2,100003,4,254350.125000,1017400.500,810000.0,22248.0,0.0,0.000000,0.0,0.000,...,0,0,0,0,0,0,0,0,0.000000,0.250000
3,100004,2,94518.900000,189037.800,94537.8,94500.0,0.0,0.000000,0.0,0.000,...,0,0,0,0,0,0,0,0,0.000000,0.000000
4,100005,3,219042.000000,657126.000,568800.0,29826.0,568408.5,189469.500000,543087.0,0.000,...,0,0,0,0,0,0,0,0,0.864992,0.666667


## 5. Merge with Main Application Table

In [ ]:
# Load application training data
application = pd.read_csv('../data/raw/application_train.csv')
print(f"Application shape: {application.shape}")
application.head()

In [14]:
# Merge bureau features with application
df = application.merge(bureau_agg, on='SK_ID_CURR', how='left')
print(f"Merged shape: {df.shape}")

# Check how many applicants had bureau data
has_bureau = df['bureau_credit_count'].notna().sum()
print(f"Applicants with bureau data: {has_bureau} ({has_bureau/len(df)*100:.1f}%)")

Merged shape: (48744, 159)
Applicants with bureau data: 42320 (86.8%)


In [15]:
# Fill NaN for applicants with no bureau history
bureau_cols = [col for col in df.columns if col.startswith('bureau_')]
df[bureau_cols] = df[bureau_cols].fillna(0)

print(f"Final merged dataframe shape: {df.shape}")
df.head()

Final merged dataframe shape: (48744, 159)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,bureau_type_loan_for_purchase_of_shares_(margin_lending),bureau_type_loan_for_the_purchase_of_equipment,bureau_type_loan_for_working_capital_replenishment,bureau_type_microloan,bureau_type_mobile_operator_loan,bureau_type_mortgage,bureau_type_real_estate_loan,bureau_type_unknown_type_of_loan,bureau_debt_credit_ratio,bureau_active_ratio
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.410555,0.428571
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.864992,0.666667
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.122498,0.416667
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


## 6. Summary of Created Features

In [16]:
print("Bureau features created:")
for col in bureau_cols:
    print(f"  - {col}")
print(f"\nTotal bureau features: {len(bureau_cols)}")

Bureau features created:
  - bureau_credit_count
  - bureau_credit_sum_mean
  - bureau_credit_sum_sum
  - bureau_credit_sum_max
  - bureau_credit_sum_min
  - bureau_debt_sum
  - bureau_debt_mean
  - bureau_debt_max
  - bureau_overdue_max
  - bureau_overdue_mean
  - bureau_days_overdue_max
  - bureau_days_credit_mean
  - bureau_days_credit_min
  - bureau_days_credit_max
  - bureau_prolong_sum
  - bureau_annuity_mean
  - bureau_annuity_sum
  - bureau_active
  - bureau_bad_debt
  - bureau_closed
  - bureau_sold
  - bureau_type_another_type_of_loan
  - bureau_type_car_loan
  - bureau_type_cash_loan_(non-earmarked)
  - bureau_type_consumer_credit
  - bureau_type_credit_card
  - bureau_type_interbank_credit
  - bureau_type_loan_for_business_development
  - bureau_type_loan_for_purchase_of_shares_(margin_lending)
  - bureau_type_loan_for_the_purchase_of_equipment
  - bureau_type_loan_for_working_capital_replenishment
  - bureau_type_microloan
  - bureau_type_mobile_operator_loan
  - bureau_ty

In [17]:
# Check feature statistics
df[bureau_cols].describe()

,bureau_credit_count,bureau_credit_sum_mean,bureau_credit_sum_sum,bureau_credit_sum_max,bureau_credit_sum_min,bureau_debt_sum,bureau_debt_mean,bureau_debt_max,bureau_overdue_max,bureau_overdue_mean,...,bureau_type_loan_for_purchase_of_shares_(margin_lending),bureau_type_loan_for_the_purchase_of_equipment,bureau_type_loan_for_working_capital_replenishment,bureau_type_microloan,bureau_type_mobile_operator_loan,bureau_type_mortgage,bureau_type_real_estate_loan,bureau_type_unknown_type_of_loan,bureau_debt_credit_ratio,bureau_active_ratio
count,48744.000000,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,4.874400e+04,...,48744.0,48744.000000,48744.000000,48744.000000,48744.0,48744.000000,48744.000000,48744.000000,48744.000000,48744.000000
mean,5.151465,3.449307e+05,1.928045e+06,9.461869e+05,9.945752e+04,6.394341e+05,1.435135e+05,4.933832e+05,3.929079e+03,2.090456e+03,...,0.0,0.000041,0.001887,0.069916,0.0,0.057115,0.000082,0.001703,0.243612,0.341301
std,4.843972,7.552857e+05,4.294585e+06,3.203559e+06,4.834983e+05,1.584764e+06,5.516854e+05,1.344474e+06,6.171512e+04,4.903584e+04,...,0.0,0.006405,0.047033,0.772049,0.0,0.254415,0.009058,0.046382,0.278996,0.308908
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,7.086938e+04,1.752038e+05,1.093500e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,1.714429e+05,8.223772e+05,3.700890e+05,2.925000e+04,1.165028e+05,2.744332e+04,9.766514e+04,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.144034,0.333333
75%,7.000000,3.714008e+05,2.306902e+06,9.900000e+05,7.484455e+04,6.587601e+05,1.250550e+05,4.592860e+05,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.429809,0.500000
max,78.000000,5.537859e+07,6.091645e+08,5.850000e+08,5.175000e+07,6.544140e+07,5.175000e+07,6.544140e+07,8.381349e+06,7.431934e+06,...,0.0,1.000000,3.000000,39.000000,0.0,4.000000,1.000000,3.000000,5.403410,1.000000
